# 第5章 誤差逆伝播法
* 前章では重みパラメータの勾配（重みパラメータに関する損失関数の勾配）を数値微分で決定した．
* 数値微分は簡単だが計算に時間がかかる
* 本章では重みパラメータの効率良い計算法として「誤差逆伝播法」について学ぶ
* 誤差逆伝播法を理解する方法
    * 数式による方法：一般的な方法．厳密で簡潔．
    * 計算グラフ（computational graph）による方法：視覚的，分かりやすい．
        * [Hacker's guide to Neural Networks](http://karpathy.github.io/neuralnets/)
        * [CS231n: Convolutional Neural Networks for Visual Recognition](http://cs231n.github.io)

## 5.1 計算グラフ
計算グラフとは，計算の過程をグラフで表したもの
グラグとは，複数のノードとエッジ（ノード間を結ぶ直線）によって表現される，データ構造としてのグラフ
### 5.1.1 計算グラフで解く
問1：太郎くんはスーパーで1個100円のリンゴを2個買いました．支払う金額を求めなさい．ただし，消費税が10％適用されるものとします．

* 計算グラフはノードと矢印で計算過程を表す．
* ノードは○で表記し，○の中に演算の内容を書く
* 計算の途中結果を矢印の上部に書くことで，ノードごとの計算結果が左から右へ伝わるように表す

<img src="./fig1.png">

```
# http://viz-js.com/
digraph G {
    rankdir=LR;
    node [shape=circle]
    n0 [label="リンゴ"]
    n1 [label="×2"]
    n2 [label="×1.1"]
    n3 [label="合計"]

    n0 -> n1 [label="100"]
    n1 -> n2 [label="200"]
    n2 -> n3 [label="220"]
}
```

* 最初にリンゴの100円が「×2」ノードへ流れ，200円になって次のノードの伝達される
* その200円が「×1.1」ノードへ流れ，220円になる
* この計算グラフの結果から答えは220円になる

#### 別の書き方

<img src="./fig2.png">

```
# http://viz-js.com/
digraph G {
    rankdir=LR;
    node [shape=circle]
    n0 [label="リンゴ"]
    n01 [label="リンゴ\nの個数"]
    n1 [label="×"]
    n11 [label="消費税"]
    n2 [label="×"]
    n3 [label="合計"]

    n0 -> n1 [label="100"]
    n01 -> n1 [label="2"]
    n1 -> n2 [label="200"]
    n11 -> n2 [label="1.1"]
    n2 -> n3 [label="220"]
}
```

問2：太郎くんはスーパーでリンゴを2個，みかんを3個買いました．リンゴは1個100円，みかんは1個150円です．消費税が10％かかるものとして，支払う金額を求めなさい．

<img src="./fig3.png">

```
# http://viz-js.com/
digraph G {
    rankdir=LR;
    node [shape=circle]
    n00 [label="リンゴ"]
    n01 [label="リンゴ\nの個数"]
    n03 [label="みかん\nの個数"]
    n02 [label="みかん"]

    n10 [label="×"]
    n11 [label="×"]
    
    n20 [label="+"]
    n21 [label="消費税"]

    n3 [label="×"]
    n4 [label="合計"]

    n00 -> n10 [label="100"]
    n01 -> n10 [label="2"]
    n02 -> n11 [label="150"]
    n03 -> n11 [label="3"]
    
    n10 -> n20 [label="200"]
    n11 -> n20 [label="450"]

    n20 -> n3 [label="650"]
    n21 -> n3 [label="1.1"]
    n3 -> n4 [label="715"]
}
```

#### 計算グラフまとめ
* 計算グラフを使って問題を解く流れ
    1. 計算グラフを構築する
    2. 計算グラフ上で計算を左から右へ進める（順伝播 forward propagation という）   


### 5.1.2 局所的な計算


### 5.1.3 なぜ計算グラフで解くのか？

## 5.2 連鎖律

### 5.2.1 計算グラフの逆伝播

### 5.2.2 連鎖律とは

### 5.2.3 連鎖律と計算グラフ

## 5.3 逆伝播

### 5.3.1 加算ノードの逆伝播

### 5.3.2 乗算ノードの逆伝播

### 5.3.3 リンゴの例

## 5.4 単純なレイヤの実装

### 5.4.1 乗算レイヤの実装

### 5.4.2 加算レイヤの実装

## 5.5 活性化関数レイヤの実装

### 5.5.1 ReLUレイヤ

### 5.5.2 Sigmoidレイヤ

## 5.6 Affine/Softmaxレイヤの実装

### 5.6.1 Affineレイヤ

### 5.6.2 バッチ版Affineレイヤ

### 5.6.3 Softmax-with-Lossレイヤ

## 5.7 誤差逆伝播法の実装

### 5.7.1 ニューラルネットワークの学習の全体図

### 5.7.2 誤差逆伝播法に対応したニューラルネットワークの実装

### 5.7.3 誤差逆伝播法の勾配確認

### 5.7.4 誤差逆伝播法を使った学習

## 5.8 まとめ

